##### Copyright 2019 The TensorFlow Authors.

In [33]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
print("imported all")

imported all


In [34]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [35]:
# Download the file
# path_to_zip = tf.keras.utils.get_file(
#     'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
#     extract=True)

# path_to_german_file = os.path.curdir+"/../data/europol-v7.de-en.de"

# path_to_english_file = os.path.curdir+"/../data/europol-v7.de-en.en"
# print(path_to_german_file)

# path_to_german_file = "C:/users/justi/Documents/RDPTranslation/initalTest/data/europarl-v7.de-en.de"

# path_to_english_file = "C:/users/justi/Documents/RDPTranslation/initalTest/data/europarl-v7.de-en.en"

path_to_german_file = "E:/RDPTranslation/initalTest/data/europarl-v7.de-en.de"

path_to_english_file = "E:/RDPTranslation/initalTest/data/europarl-v7.de-en.en"

print(path_to_german_file)

E:/RDPTranslation/initalTest/data/europarl-v7.de-en.de


In [36]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [37]:
def join_sentences(english_data, german_data):
      germ_lines = io.open(german_data, encoding='UTF-8').read().strip().split('\n')
      eng_lines = io.open(english_data, encoding='UTF-8').read().strip().split('\n')
      german_result= []
      english_result = []
      combined_data = []
      for i in range (1,30000): ## only run on first 30k lines
        german_lines_clean = preprocess_sentence(germ_lines[i])
        english_lines_clean = preprocess_sentence(eng_lines[i])
        combined_data.append(english_lines_clean)
#         combined_data.append('\t')
        combined_data.append(german_lines_clean)
        german_result.append(german_lines_clean)
        english_result.append(english_lines_clean)
      return combined_data, german_result, english_result
data, german_data, english_data = join_sentences(path_to_english_file,path_to_german_file)

In [38]:
print(len(data))
# data = data.strip('\t')
x = data[0::1]
print(len(x))

# for i in range(5):
#     print(x[i])
# print(data[3])
# print(x[3])
# print(german_data[0])
print(len(german_data))
# print(german_data)
print(english_data[0])

59998
59998
29999
<start> i declare resumed the session of the european parliament adjourned on friday december , and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period . <end>


In [39]:
joined_data = ['\t'.join(x) for x in zip(data[0::2], x[1::2])]
print(joined_data[0])
# print(len(joined_data))
print(english_data[-1])

<start> i declare resumed the session of the european parliament adjourned on friday december , and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period . <end>	<start> ich erklare die am freitag , dem . dezember unterbrochene sitzungsperiode des europaischen parlaments fur wiederaufgenommen , wunsche ihnen nochmals alles gute zum jahreswechsel und hoffe , da sie schone ferien hatten . <end>
<start> we hope that this amendment will be adopted . <end>


In [40]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [41]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,dtype='float32', maxlen=179,
                                                         padding='post')

  return tensor, lang_tokenizer

In [42]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
#   targ_lang, inp_lang = create_dataset(path, num_examples)
  
  input_tensor, inp_lang_tokenizer = tokenize(english_data)
  target_tensor, targ_lang_tokenizer = tokenize(german_data)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [43]:
input_tensor, inp_lang_tokenizer = tokenize(english_data)
target_tensor, targ_lang_tokenizer = tokenize(german_data)
print(input_tensor.shape)
# print(len(inp_lang_tokenizer))
print(target_tensor.shape)
# print(len(targ_lang_tokenizer))

(29999, 179)
(29999, 179)


### Limit the size of the dataset to experiment faster (optional)

Training on the complete dataset of >100,000 sentences will take a long time. To train faster, we can limit the size of the dataset to 30,000 sentences (of course, translation quality degrades with less data):

In [44]:
# Try experimenting with the size of that dataset
num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_english_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [45]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

23999 23999 6000 6000


In [46]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [47]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
4 ----> <start>
1 ----> the
9168 ----> fvo
11 ----> is
110 ----> made
88 ----> up
6 ----> of
765 ----> independent
2235 ----> scientists
8 ----> and
6315 ----> vets
8 ----> and
66 ----> so
18 ----> on
84 ----> who
2806 ----> travel
44 ----> from
89 ----> some
3779 ----> location
127 ----> where
45 ----> there
11 ----> is
34 ----> an
4435 ----> airport
288 ----> quite
283 ----> clearly
14 ----> we
24 ----> have
34 ----> an
4435 ----> airport
9 ----> in
2238 ----> dublin
3 ----> .
5 ----> <end>

Target Language; index to word mapping
4 ----> <start>
28 ----> dem
2975 ----> lebensmittel
7 ----> und
11088 ----> veterinaramt
750 ----> gehoren
2858 ----> unabhangige
2973 ----> wissenschaftler
1 ----> ,
19707 ----> veterinarmediziner
1020 ----> usw
3 ----> .
45 ----> an
1 ----> ,
2 ----> die
17 ----> fur
89 ----> ihre
19708 ----> reisetatigkeit
20 ----> auf
52 ----> einen
5653 ----> flughafen
2350 ----> angewiesen
32 ----> sind
1 ----> ,
7 ----> und
52 --

### Create a tf.data dataset

In [48]:
BUFFER_SIZE = len(input_tensor_train)
# BATCH_SIZE = 64
## for custom model use
BATCH_SIZE = 179
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [49]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([179, 179]), TensorShape([179, 179]))

## Define the optimizer and the loss function

In [50]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.Adam(clipnorm=1,learning_rate=lr_schedule)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [55]:
def buildModel():
    model = tf.keras.Sequential(layers=[
        # tf.keras.layers.LSTM(64),
        # tf.keras.layers.LSTM(16),
#         tf.keras.layers.Conv1D(668,kernel_size=1),
        tf.keras.layers.Conv1D(179,kernel_size=1),
        tf.keras.layers.Conv1D(64,kernel_size=1),
        tf.keras.layers.Conv1D(16,kernel_size=1),
#         tf.keras.layers.Flatten(),
        # tf.keras.layers.Conv1D(64,kernel_size=1), #input_shape=(64,668)
        # tf.keras.layers.MaxPool1D(),
        # tf.keras.layers.Conv1D(1,kernel_size=1),
        # tf.keras.layers.Dense(16),
        # # # tf.keras.layers.MaxPool1D(),
        # #
        tf.keras.layers.Dense(16),
        tf.keras.layers.Dense(64),
        # tf.keras.layers.Dense(179),
        # tf.keras.layers.Dense(45342),
        tf.keras.layers.Dense(668),
        # tf.keras.layers.Dense(668),
        # tf.keras.layers.Dense(4295821)

        # tf.keras.layers.Conv1DTranspose(1,kernel_size=1),
        # tf.keras.layers.Conv1DTranspose(668,kernel_size=1)

        ])
    return model
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


# seq2seq.summary()

# with tf.device('/cpu:0'): ## comment out to run on gpu
#     start_time = time.time()
#     seq2seq = buildModel()
#     print("Time required to create model --- %s seconds ---" % (time.time() - start_time))

#     ## Train model and print results
#     seq2seq.compile(optimizer= 'adam',loss=loss_function)
#     seq2seq.build(input_shape=(668,64,1))
#     # seq2seq.summary()
#     start_time = time.time()
#     for (batch, (inp, targ)) in enumerate(dataset.take(32)):
#         inp = tf.reshape(inp,[inp.shape[0],inp.shape[1],1])
#         targ = tf.reshape(targ,[targ.shape[0],targ.shape[1],1])
#         seq2seq.train_on_batch(x=inp,y=targ)
#         print(batch)
#         print("Time taken to train batch: %s", time.time()-start_time)
#     #     print(targ.shape)
#     #     seq2seq.fit(x=input_tensor_train,y=target_tensor_train,epochs=10)
#     seq2seq.summary()
#     seq2seq.save(filepath="./models/dense")
start_time = time.time()
seq2seq = buildModel()
print("Time required to create model --- %s seconds ---" % (time.time() - start_time))
## Train model and print results
### Loss functions
## MSE gives high error
## SparseCategoricalCrossentropy gives nan
## loss_function gives shape error
## sparse_categorical_crossentropy give shape error
print("y train")
print(target_tensor_train.shape)
print("Y val")
print(target_tensor_val.shape)
def loss_function(y_pred, y):
 
    #shape of y [batch_size, ty]
    #shape of y_pred [batch_size, Ty, output_vocab_size] 
    sparsecategoricalcrossentropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                                                  reduction='none')
    loss = sparsecategoricalcrossentropy(y_true=y, y_pred=y_pred)
    mask = tf.logical_not(tf.math.equal(y,0))   #output 0 for y=0 else output 1
    mask = tf.cast(mask, dtype=loss.dtype)
    loss = mask* loss
    loss = tf.reduce_mean(loss)
    return loss
seq2seq.compile(optimizer=optimizer,loss='mean_squared_logarithmic_error',metrics= ['accuracy'])
seq2seq.build(input_shape=(668,64,1))
# seq2seq.summary()
start_time = time.time()
seq2seq.summary()
for (batch, (inp, targ)) in enumerate(dataset.take(64)):
    inp = tf.reshape(inp,[inp.shape[0],inp.shape[1],1])
    targ = tf.reshape(targ,[targ.shape[0],targ.shape[1],1])
    loss = seq2seq.train_on_batch(x=inp,y=targ)
    print(batch)
    print("Time taken to train batch: %s", time.time()-start_time)
    print("batch loss: %f", loss)
    print(targ.shape)
    seq2seq.fit(x=inp,y=targ,epochs=5)
seq2seq.save(filepath="./models/nmt")    

==============] - 0s 7ms/step - loss: 2.2823 - accuracy: 3.1210e-05
Epoch 5/5
6/6 [==============================] - 0s 9ms/step - loss: 2.2815 - accuracy: 1.8726e-04
32
Time taken to train batch: %s 12.859038829803467
batch loss: %f [2.1169726848602295, 0.0001092350430553779]
(179, 179, 1)
Epoch 1/5
6/6 [==============================] - 0s 9ms/step - loss: 1.9530 - accuracy: 3.1210e-05
Epoch 2/5
6/6 [==============================] - 0s 7ms/step - loss: 1.9492 - accuracy: 9.3630e-05
Epoch 3/5
6/6 [==============================] - 0s 8ms/step - loss: 1.9478 - accuracy: 6.2420e-05
Epoch 4/5
6/6 [==============================] - 0s 7ms/step - loss: 1.9489 - accuracy: 6.2420e-05
Epoch 5/5
6/6 [==============================] - 0s 7ms/step - loss: 1.9499 - accuracy: 3.1210e-05
33
Time taken to train batch: %s 13.195049524307251
batch loss: %f [2.137209415435791, 3.12100128212478e-05]
(179, 179, 1)
Epoch 1/5
6/6 [==============================] - 0s 8ms/step - loss: 2.3232 - accuracy: 3.

## Translate

* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token*.
* And store the *attention weights for every time step*.

Note: The encoder output is calculated only once for one input.

In [52]:
ge_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
ge_tokenizer.fit_on_texts(german_data)

In [57]:
for (batch, (inp, targ)) in enumerate(dataset.take(16)):
    # print(inp.shape)
    # print(targ.shape)
    inp = tf.reshape(inp,[inp.shape[0],inp.shape[1],1])
    targ = tf.reshape(targ,[targ.shape[0],targ.shape[1],1])
    predictions = []
    predictions.append(seq2seq.predict_on_batch(inp))
    german_output = ge_tokenizer.sequences_to_texts(predictions[0][batch-1])
    # print(german_output.shape)
    while("" in german_output) : 
        german_output.remove('')
    print(german_output)
    # print(predictions)
# print(predictions[0])
results = predictions[0]
# german_output = ge_tokenizer.sequences_to_texts(results)
# print(results.shape)
# print(results[1][1])
# output = results[1][1]

['schones preises']
[]
['ausgewogene']
['frage', 'ii']
['zweiten']
[]
['abstimmen']
['bildungs']
[]
['mir', 'demokratischen', 'wichtigsten', 'demokratischen']
['luftschadstoffe']
[]
[]
['mir']
[]
['mogliche', 'herren']


In [54]:
# def translate(sentence):
#   inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
#   inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
#                                                          maxlen=max_length_inp,

#                                                          padding='post')
#   inputs = tf.convert_to_tensor(inputs,dtype='float32')
#   print(inputs.shape)
#   inputs = input_tensor_val
#   inputs = tf.reshape(inputs,[inputs.shape[0],inputs.shape[1],1])
#   print(inputs.shape)
  
#   output=seq2seq.predict(x=inputs)

#   # convert(targ_lang,output.all())
# #   output=tf.convert_to_tensor(output,dtype='float32')
#   # print(output[0].shape)
# #   print(output.type)
#   # print(output[0])
#   x = output[0]
#   print(x)

 
# #   output_sample = output.tolist()
#   ge_tokenizer = tf.keras.preprocessing.text.Tokenizer(
#       filters='')
#   ge_tokenizer.fit_on_texts(german_data)
#   german_output = ge_tokenizer.sequences_to_texts(x)
#   print(german_output)
# #   print(*(i for i in german_output),sep="\n")
# translate("i declare resumed the session of the european parliament adjourned on friday december")